# ESTIMATING 1-MONTH OPTION PRICE ON NIFTY USING MONTE-CARLO SIMULATION

In [2]:
import pandas as pd
import numpy as np
import random
import yfinance as yf

 We use the price data for the month of August to calculate the *mean* and *SD* estimates,
 to be used in the Simulation exercise to forecast Price paths over the next month.

- Price Calculated on : **01-09-2024**
- Option Expiry Date : **03-10-2024**
- Number of Trading days = **23**

In [3]:
data = yf.download('^NSEI',
                   start ="2024-08-01",
                   end = "2024-08-31",
                   progress = False)
data1 = data.copy(deep=True)

In [4]:
price = data1["Adj Close"]
returns = np.log(price/price.shift(1)).dropna()

In [5]:
mu = returns.mean()
sd = returns.std()
print("Mean and SD estimates of the returns of NIFTY index in the month of August are", mu ,"and",sd,"respectively")

Mean and SD estimates of the returns of NIFTY index in the month of August are 0.00044779269602611275 and 0.009365599456199991 respectively


## <u>ASSUMPTIONS<u>

- We assume that the stocks in the NIFTY index follows a **<u>Geometric Brownian Motion<u>**
- Extending the above assumption for the index, we will assume that the **NIFTY also
   follows a Geometric Brownian Motion(GBM)**
- We use the closed form solution of the GBM to generate price paths for the NIFTY
   over the next month( i.e. 23 trading days),
   we will simulate 100 Price Paths.
- GBM follows a Stochastic Differntial Equation
   **$dS(t) = S(t)(mu * dt + sd*dW(t))$**
   where
   - dt : timesteps, in our case dt=1
   - dW(t) ~ N(0,dt) and dW(t) = W(t)-W(t-1)
      
- Closed Form Solution to the above SDE is,

    **$S(t) = S(0)*exp ^(mu - 0.5 * sd^2)* t + sd * W(t)$**
   where
   - S(t) : Value of the process at time,t.
   - S(0) : Value of the process at time 0 or the recent traded price on the exchange.
   - W(t) : Value of the Weiner process at time,t 

- We use the 10 year Govt Bond yield as a proxy for the risk free rate,
   **risk free rate = 7.365% p.a** 

- A **seed value=100** is used so that the process generates a unique answer.

## <u>FUNCTION TO CALCULATE OPTION PRICE<u>

- We assume Continuous compounding

- **Price of the Option = Mean of Present Value of all the Simulated Option Payoffs.**

- Strike Price, K=**25200**

- Time steps = 1,
  that is we simulate end of day closing price for 23 days.
 
- tenure = days to expiry 

In [8]:
def MCS_OptionPrice(pricepaths,n_simulation,S0,strike,rfr,tenure,mean,sigma):
    dt=1
    dW = np.zeros((pricepaths,n_simulation))
    np.random.seed(100)
    for i in range(0,pricepaths):
        for j in range(0,n_simulation):
             a = np.random.normal(loc=0,scale=np.sqrt(dt))
             dW[i][j] = a
    W = np.cumsum(dW,axis=1)
    t = np.linspace(1,n_simulation,n_simulation)
    t = np.broadcast_to(t,(pricepaths,n_simulation))
    St = S0 * np.exp((mu - 0.5*sigma**2)*t + sigma*W)
    St = pd.DataFrame(St)
    terminal_price = St.iloc[:,-1]
    payoff_call = np.maximum(0,terminal_price - strike)
    payoff_put = np.maximum(strike - terminal_price,0)
    pv_c = payoff_call * np.exp(-(rfr/365) * tenure)
    Call_price = round(pv_c.mean(),3)
    pv_p = payoff_put * np.exp(-(rfr/365) * tenure)
    Put_price = round(pv_p.mean(),3)
    return Call_price,Put_price

## CALCULATING OPTION PRICE - CALL AND PUT

In [10]:
Current_price = price.iloc[-1]

In [11]:
Optionprice = MCS_OptionPrice(100,23,Current_price,25200,0.07365,32,mu,sd)
print("Call Option Price and Put Option Price are", Optionprice , " respectively")

Call Option Price and Put Option Price are (626.64, 280.806)  respectively


## <u>CONCLUSION<u>

## CALL-OPTION

1. Market Price of Call Option on NIFTY for 1 month maturity and is 494.15 INR
2. Simulated Call Option Price on NIFTY for 1 month maturity is 626.64 INR

- We assume that the  Simulated Result is the Fair Value estimate of the Option Price.
   Since Fair Value > Market Value

   **<u>DECISION<u>**:
   *We should Buy the Option and sell the option sometime before it matures* assuming that
   
   we have enough volatility,given a small time frame of 1 month, such that Market Value increases and converges to the Fair Value.

## PUT-OPTION

In [14]:
print("Market Price of Put Option on NIFTY for 1 month maturity is 279.90 INR")
print("Simulated Put Option Price on NIFTY for 1 month maturity is 280.80 INR")

Market Price of Put Option on NIFTY for 1 month maturity is 279.90 INR
Simulated Put Option Price on NIFTY for 1 month maturity is 280.80 INR


- Same Assumption as Above,
   Here Fair Value is almost equal to Market Value.

   **<u>DECISION<u>**:
   *Neither Buy nor Sell*, assuming that volatility does not fluctuate enough for a material change
   in Price.

   Since, Time left to expiry is only a month,it would be rational to assume that market would'nt display
   any significant volatility.